<table border="0" style="width:100%">
 <tr>
    <td>
        <img src="https://static-frm.ie.edu/university/wp-content/uploads/sites/6/2022/06/IE-University-logo.png" width=150>
     </td>
    <td><div style="font-family:'Courier New'">
            <div style="font-size:25px">
                <div style="text-align: right">
                    <b> MASTER IN BIG DATA</b>
                    <br>
                    Python for Data Analysis II
                    <br><br>
                    <em> Daniel Sierra Ramos </em>
                </div>
            </div>
        </div>
    </td>
 </tr>
</table>

In [53]:
import json
import zipfile
import geopandas

import numpy as np
import pandas as pd

import plotly.express as px

import streamlit as st

## **Streamlit** App with Uber data

Build a Streamlit app to represent some charts about using public Uber data. This data contains information about the average travel time of Uber rides between all neighborhoods in MAdrid city.

The app must have the following characteristics
 - Main frame must show 3 charts given a `SOURCE` and `DESTINATION`:
    1. Time series of the average travel time
    2. Bar chart of the average travel time for every weekday, and period of the day
    3. **(Optional)** Choropleth map of Madrid city, with the source and destination highlighted
 - A side bar containing two box selectors to select the `SOURCE` and `DESTINATION`
 - Every time a source or a destination is selected, the charts of the main frame should be updated accordingly.

 NOTE: I recommend you to build the charts first here in the notebook, and then copy the code to the Streamlit app.

### Load data

Use the following function to load the data. For the first two figures you will need just the `data` variable. For the choropleth map you will need the `codes` variable and the `geopandas` library to work with maps (this is optional).

In [54]:
def read_and_preprocess_data():

    with zipfile.ZipFile('data/uber-data.zip') as zip:
        with zip.open('madrid-barrios-2020-1-All-DatesByHourBucketsAggregate.csv') as csv:
            data = pd.read_csv(csv)
        with zip.open('madrid_barrios.json') as geojson:
            codes = geopandas.read_file(geojson, encoding="utf-8")

    # change data types in codes because they are not the same as in data
    codes['GEOCODIGO'] = codes['GEOCODIGO'].astype(int)
    codes['MOVEMENT_ID'] = codes['MOVEMENT_ID'].astype(int)

    codes["DISPLAY_NAME"] = codes["DISPLAY_NAME"].str.split().str[1:].str.join(" ")

    # Merge the data with the codes (source)
    data = data.merge(codes[["GEOCODIGO","MOVEMENT_ID","DISPLAY_NAME"]], left_on="sourceid", right_on="MOVEMENT_ID", how="left")
    data = data.rename(columns={"GEOCODIGO":"src_neigh_code", "DISPLAY_NAME":"src_neigh_name"}).drop(columns=["MOVEMENT_ID"])

    data = data.merge(codes[["GEOCODIGO","MOVEMENT_ID","DISPLAY_NAME"]], left_on="dstid", right_on="MOVEMENT_ID", how="left")
    data = data.rename(columns={"GEOCODIGO":"dst_neigh_code", "DISPLAY_NAME":"dst_neigh_name"}).drop(columns=["MOVEMENT_ID"])

    # Create a new date column
    data["year"] = "2020"
    data["date"] = pd.to_datetime(data["day"].astype(str)+data["month"].astype(str)+data["year"].astype(str)+":"+data["start_hour"].astype(str), format="%d%m%Y:%H")

    # Create a new day_period column
    data["day_period"] = data.start_hour.astype(str) + "-" + data.end_hour.astype(str)
    data["day_of_week"] = data.date.dt.weekday
    data["day_of_week_str"] = data.date.dt.day_name()

    return data, codes

### Travel by Time and Day

In [55]:
data, codes = read_and_preprocess_data()

In [56]:
data.head(20)

,sourceid,dstid,month,day,start_hour,end_hour,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,src_neigh_code,src_neigh_name,dst_neigh_code,dst_neigh_name,year,date,day_period,day_of_week,day_of_week_str
0,75,12,1,25,10,16,838.25,324.30,776.22,1.49,79123,San Fermín,79026,Palos de Moguer,2020,2020-01-25 10:00:00,10-16,5,Saturday
1,20,19,1,23,7,10,567.20,278.07,504.47,1.63,79041,Recoletos,79036,Niño Jesús,2020,2020-01-23 07:00:00,7-10,3,Thursday
2,52,9,2,8,0,7,470.88,268.89,412.66,1.64,79091,Casa de Campo,79023,La Chopera,2020,2020-02-08 00:00:00,0-7,5,Saturday
3,20,19,2,13,7,10,658.88,569.66,524.05,1.82,79041,Recoletos,79036,Niño Jesús,2020,2020-02-13 07:00:00,7-10,3,Thursday
4,14,25,1,26,10,16,519.17,240.26,478.95,1.46,79031,Pacífico,79046,Castellana,2020,2020-01-26 10:00:00,10-16,6,Sunday
5,80,124,1,2,16,19,696.37,354.81,640.21,1.44,79131,Entrevías,79206,Rejas,2020,2020-01-02 16:00:00,16-19,3,Thursday
6,52,9,1,18,0,7,470.06,229.17,422.46,1.58,79091,Casa de Campo,79023,La Chopera,2020,2020-01-18 00:00:00,0-7,5,Saturday
7,70,62,1,25,10,16,430.79,183.51,390.22,1.58,79115,Puerta Bonita,79104,Aluche,2020,2020-01-25 10:00:00,10-16,5,Saturday
8,70,62,2,15,10,16,450.68,195.78,412.38,1.52,79115,Puerta Bonita,79104,Aluche,2020,2020-02-15 10:00:00,10-16,5,Saturday
9,51,16,1,1,10,16,923.14,454.90,843.03,1.49,79088,El Goloso,79033,Estrella,2020,2020-01-01 10:00:00,10-16,2,Wednesday


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2285185 entries, 0 to 2285184
Data columns (total 19 columns):
 #   Column                                    Dtype         
---  ------                                    -----         
 0   sourceid                                  int64         
 1   dstid                                     int64         
 2   month                                     int64         
 3   day                                       int64         
 4   start_hour                                int64         
 5   end_hour                                  int64         
 6   mean_travel_time                          float64       
 7   standard_deviation_travel_time            float64       
 8   geometric_mean_travel_time                float64       
 9   geometric_standard_deviation_travel_time  float64       
 10  src_neigh_code                            int64         
 11  src_neigh_name                            object        
 12  dst_neigh_code

In [58]:
origin = data.src_neigh_name.sort_values().unique()
destination = data.dst_neigh_name.unique()

origin

array(['Abrantes', 'Adelfas', 'Aeropuerto', 'Alameda de Osuna', 'Almagro',
       'Almenara', 'Almendrales', 'Aluche', 'Amposta', 'Apóstol Santiago',
       'Arapiles', 'Aravaca', 'Arcos', 'Argüelles', 'Atalaya', 'Atocha',
       'Bellas Vistas', 'Berruguete', 'Buenavista', 'Butarque',
       'Campamento', 'Canillas', 'Canillejas', 'Casa de Campo',
       'Casco Histórico de Barajas', 'Casco Histórico de Vallecas',
       'Casco Histórico de Vicálvaro', 'Castellana', 'Castilla',
       'Castillejos', 'Ciudad Jardín', 'Ciudad Universitaria', 'Colina',
       'Comillas', 'Concepción', 'Corralejos', 'Cortes', 'Costillares',
       'Cuatro Caminos', 'Cuatro Vientos', 'Delicias', 'El Cañaveral',
       'El Goloso', 'El Pardo', 'El Pilar', 'El Plantío', 'El Salvador',
       'El Viso', 'Embajadores', 'Ensanche de Vallecas', 'Entrevías',
       'Estrella', 'Fontarrón', 'Fuente del Berro', 'Fuentelarreina',
       'Gaztambide', 'Goya', 'Guindalera', 'Hellín', 'Hispanoamérica',
       'Horcajo'

In [59]:
#plot average travel time by date, for a given source and destination

In [60]:

    # group data by date, keep all columns
    data_grouped = data.groupby("date").agg({"mean_travel_time":"mean"})
    # plot
    data_grouped.mean_travel_time = data_grouped.mean_travel_time.astype(int)/60

    data_grouped

,mean_travel_time
date,
2020-01-01 00:00:00,16.016667
2020-01-01 07:00:00,11.066667
2020-01-01 10:00:00,10.666667
2020-01-01 16:00:00,9.000000
2020-01-01 19:00:00,9.833333
...,...
2020-03-31 00:00:00,5.200000
2020-03-31 07:00:00,5.583333
2020-03-31 10:00:00,6.233333


### Travel by Time and Day Period

### Map between source and destination

In [61]:
import plotly.express as px
data_filtered = data[(data.src_neigh_name == "Aeropuerto") & (data.dst_neigh_name == "Recoletos")]

# group data by date, keep all columns
data_grouped = data.groupby("date").agg({"sourceid":"count"}, as_index=False).rename(columns={"sourceid":"trips"})

# merge
data_grouped = data_grouped.merge(data[["date","src_neigh_name","dst_neigh_name"]], on="date", how="left")


In [62]:
data_filtered

,sourceid,dstid,month,day,start_hour,end_hour,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,src_neigh_code,src_neigh_name,dst_neigh_code,dst_neigh_name,year,date,day_period,day_of_week,day_of_week_str
63880,128,20,1,8,16,19,1521.69,337.01,1484.19,1.25,79212,Aeropuerto,79041,Recoletos,2020,2020-01-08 16:00:00,16-19,2,Wednesday
75733,128,20,1,17,16,19,1410.27,550.65,1341.88,1.34,79212,Aeropuerto,79041,Recoletos,2020,2020-01-17 16:00:00,16-19,4,Friday
75734,128,20,2,7,16,19,1376.25,242.59,1357.33,1.18,79212,Aeropuerto,79041,Recoletos,2020,2020-02-07 16:00:00,16-19,4,Friday
88122,128,20,2,27,19,0,1344.55,525.71,1251.98,1.45,79212,Aeropuerto,79041,Recoletos,2020,2020-02-27 19:00:00,19-0,3,Thursday
133603,128,20,3,2,7,10,1211.89,396.03,1157.03,1.34,79212,Aeropuerto,79041,Recoletos,2020,2020-03-02 07:00:00,7-10,0,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222651,128,20,2,12,10,16,1127.31,257.81,1101.15,1.24,79212,Aeropuerto,79041,Recoletos,2020,2020-02-12 10:00:00,10-16,2,Wednesday
2246179,128,20,2,8,16,19,1014.30,210.94,993.07,1.23,79212,Aeropuerto,79041,Recoletos,2020,2020-02-08 16:00:00,16-19,5,Saturday
2246183,128,20,1,18,16,19,1214.52,551.07,1117.41,1.48,79212,Aeropuerto,79041,Recoletos,2020,2020-01-18 16:00:00,16-19,5,Saturday
2246391,128,20,1,3,19,0,1050.45,328.40,963.28,1.72,79212,Aeropuerto,79041,Recoletos,2020,2020-01-03 19:00:00,19-0,4,Friday


In [69]:
# Bar chart of the average travel time for every weekday, and period of the day
data_grouped = data.groupby(["day_of_week_str","day_period"]).agg({"mean_travel_time":"mean"}).reset_index()
data_grouped.mean_travel_time = data_grouped.mean_travel_time.astype(int)/60
data_grouped

,day_of_week_str,day_period,mean_travel_time
0,Friday,0-7,8.000000
1,Friday,10-16,10.800000
2,Friday,16-19,9.433333
3,Friday,19-0,9.716667
4,Friday,7-10,9.916667
5,Monday,0-7,8.166667
6,Monday,10-16,9.100000
7,Monday,16-19,9.033333
8,Monday,19-0,8.750000
9,Monday,7-10,10.866667


In [63]:
# get codes where DISPLAY_NAME is Aeropuerto and Recoletos
src_dst = codes[(codes.DISPLAY_NAME == "Aeropuerto") | (codes.DISPLAY_NAME == "Recoletos")]

src_dst

,CODBDT,GEOCODIGO,MOVEMENT_ID,DISPLAY_NAME,geometry
19,860664,79041,20,Recoletos,"POLYGON ((-3.69247 40.41947, -3.69267 40.41945..."
127,860772,79212,128,Aeropuerto,"POLYGON ((-3.54210 40.49437, -3.54197 40.49428..."


In [64]:
import streamlit as st
import plotly.express as px

# Define the two locations
locations = {'New York City': [40.7128, -74.0060], 'London': [51.5074, -0.1278]}

# Create a Plotly figure object with a map
fig = px.scatter_mapbox(locations, lat=[loc[0] for loc in locations.values()], lon=[loc[1] for loc in locations.values()], hover_name=list(locations.keys()), zoom=2)

# Add markers for the two locations
fig.add_scattermapbox(lat=[locations['New York City'][0], locations['London'][0]], lon=[locations['New York City'][1], locations['London'][1]], mode='markers', marker=dict(color='red', size=10))

# Set the layout of the figure
fig.update_layout(mapbox_style='open-street-map', margin=dict(l=0, r=0, t=0, b=0))

# Display the map in Streamlit using the Plotly PlotlyChart component
st.set_page_config(layout="wide")
st.title('Map with Two Locations')
st.plotly_chart(fig)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)